# Deep Dive: The `config` Parameter in `call_model`

Let me explain the `config: RunnableConfig` parameter in detail, as it's a crucial but often overlooked component that controls how the agent executes.

## What is RunnableConfig?

```python
def call_model(
    state: ChatAgentState,
    config: RunnableConfig,  # ← This parameter
):
    response = model_runnable.invoke(state, config)
    return {"messages": [response]}
```

`RunnableConfig` is LangChain's way of passing **execution-time configuration** and **runtime context** to any runnable component. Think of it as a "control panel" for how your agent should behave during execution.

---

## RunnableConfig Structure

```python
from langchain_core.runnables import RunnableConfig

config = RunnableConfig(
    # Core configuration
    configurable: Dict[str, Any] = {},
    
    # Observability and monitoring
    callbacks: List[BaseCallbackHandler] = [],
    metadata: Dict[str, Any] = {},
    tags: List[str] = [],
    
    # Execution control
    recursion_limit: int = 25,
    max_concurrency: Optional[int] = None,
    
    # Threading and async
    thread_id: Optional[str] = None,
    checkpoint_id: Optional[str] = None,
)
```

Let me break down each component:

---

## 1. Configurable Parameters

```python
config = RunnableConfig(
    configurable={
        "temperature": 0.1,
        "max_tokens": 500,
        "top_p": 0.9,
        "custom_param": "value"
    }
)
```

### What it does:
- **Runtime overrides**: Change model behavior without modifying the base configuration
- **Per-request customization**: Different users can have different settings
- **A/B testing**: Test different configurations dynamically

### Example usage in our Databricks agent:

```python
# Base agent configuration
baseline_config = {
    "endpoint_name": "databricks-meta-llama-3-3-70b-instruct",
    "temperature": 0.01,
    "max_tokens": 1000,
}

# Runtime override for creative tasks
creative_config = RunnableConfig(
    configurable={
        "temperature": 0.8,  # More creative responses
        "max_tokens": 2000   # Longer responses
    }
)

# Runtime override for analytical tasks  
analytical_config = RunnableConfig(
    configurable={
        "temperature": 0.0,  # Deterministic responses
        "max_tokens": 500    # Concise responses
    }
)

# Use different configs for different scenarios
response = agent.invoke(messages, config=creative_config)
```

### How it works in practice:

```python
def _build_agent_from_config(self):
    llm = ChatDatabricks(
        endpoint=self.config.get("endpoint_name"),
        temperature=self.config.get("temperature"),  # Can be overridden
        max_tokens=self.config.get("max_tokens"),    # Can be overridden
    )
    # ...
```

When `call_model` executes:
```python
# The configurable parameters merge with base config
effective_temperature = config.configurable.get("temperature") or base_config["temperature"]
```

---

## 2. Callbacks for Observability

```python
config = RunnableConfig(
    callbacks=[
        StdOutCallbackHandler(),        # Print to console
        MLflowCallbackHandler(),        # Log to MLflow
        CustomMetricsHandler(),         # Custom monitoring
    ]
)
```

### What callbacks enable:

**Real-time monitoring:**
```python
class ConversationMonitor(BaseCallbackHandler):
    def on_llm_start(self, serialized, prompts, **kwargs):
        print(f"🤖 LLM called with {len(prompts)} prompts")
        
    def on_llm_end(self, response, **kwargs):
        print(f"✅ LLM responded with {len(response.generations)} outputs")
        
    def on_tool_start(self, serialized, input_str, **kwargs):
        print(f"🔧 Tool {serialized['name']} called")
        
    def on_tool_end(self, output, **kwargs):
        print(f"✅ Tool completed: {output[:100]}...")
```

**Performance tracking:**
```python
class PerformanceTracker(BaseCallbackHandler):
    def __init__(self):
        self.start_time = None
        self.token_count = 0
        
    def on_llm_start(self, **kwargs):
        self.start_time = time.time()
        
    def on_llm_end(self, response, **kwargs):
        duration = time.time() - self.start_time
        # Log metrics to your monitoring system
        log_metric("llm_duration", duration)
        log_metric("token_count", self.token_count)
```

**Usage in our agent:**
```python
# Monitor a specific conversation
monitoring_config = RunnableConfig(
    callbacks=[ConversationMonitor(), PerformanceTracker()]
)

# Every call_model execution will trigger these callbacks
response = call_model(state, monitoring_config)
```

---

## 3. Metadata for Context

```python
config = RunnableConfig(
    metadata={
        "user_id": "sourav_banerjee",
        "session_id": "session_123",
        "request_type": "documentation_query",
        "priority": "high",
        "source": "web_interface",
        "experiment_group": "feature_test_a"
    }
)
```

### How metadata is used:

**Request tracking:**
```python
def call_model(state: ChatAgentState, config: RunnableConfig):
    user_id = config.metadata.get("user_id", "anonymous")
    session_id = config.metadata.get("session_id", "unknown")
    
    # Log request with context
    logger.info(f"Processing request for user {user_id} in session {session_id}")
    
    response = model_runnable.invoke(state, config)
    return {"messages": [response]}
```

**Personalization:**
```python
def personalized_preprocessing(state, config):
    user_id = config.metadata.get("user_id")
    user_preferences = get_user_preferences(user_id)
    
    # Modify system prompt based on user preferences
    if user_preferences.get("technical_level") == "expert":
        system_prompt = "Provide detailed technical responses..."
    else:
        system_prompt = "Provide beginner-friendly responses..."
    
    return enhanced_state
```

**Analytics and A/B testing:**
```python
def analytics_callback(config):
    experiment_group = config.metadata.get("experiment_group")
    
    # Track metrics by experiment group
    track_metric(f"response_time_{experiment_group}", duration)
    track_metric(f"tool_usage_{experiment_group}", tool_count)
```

---

## 4. Tags for Categorization

```python
config = RunnableConfig(
    tags=[
        "production",
        "databricks_docs",
        "technical_support",
        "priority_user"
    ]
)
```

### Tag usage examples:

**Filtering and monitoring:**
```python
class TaggedMetricsHandler(BaseCallbackHandler):
    def on_llm_end(self, response, **kwargs):
        tags = kwargs.get("tags", [])
        
        if "priority_user" in tags:
            # Send to priority monitoring dashboard
            send_to_priority_dashboard(response)
            
        if "production" in tags:
            # Different SLA requirements
            check_response_quality(response)
```

**Configuration routing:**
```python
def get_model_config(tags):
    if "high_accuracy" in tags:
        return {"temperature": 0.0}
    elif "creative" in tags:
        return {"temperature": 0.8}
    else:
        return {"temperature": 0.1}
```

---

## 5. Execution Control Parameters

### Recursion Limit
```python
config = RunnableConfig(
    recursion_limit=10  # Prevent infinite loops
)
```

**Why this matters:**
```python
# Without recursion limit, this could loop forever:
# Agent calls tool → Tool calls another tool → Tool calls agent → Agent calls tool...

# With recursion limit:
# After 10 levels deep, execution stops with an error
```

**Custom recursion handling:**
```python
def safe_call_model(state, config):
    try:
        return model_runnable.invoke(state, config)
    except RecursionError:
        return {
            "role": "assistant", 
            "content": "I apologize, this query is too complex. Could you break it down into smaller parts?"
        }
```

### Max Concurrency
```python
config = RunnableConfig(
    max_concurrency=5  # Limit parallel operations
)
```

**Use case:**
```python
# If agent calls multiple tools simultaneously:
tools = ["search_docs", "query_database", "call_api", "analyze_data"]

# Without limit: All 4 tools run in parallel (might overwhelm system)
# With limit: Only 5 concurrent operations allowed
```

---

## 6. Threading and Checkpointing

```python
config = RunnableConfig(
    thread_id="conversation_abc123",
    checkpoint_id="checkpoint_001"
)
```

### Thread ID usage:
```python
# Maintains conversation context across multiple calls
thread_id = f"user_{user_id}_session_{session_id}"

config = RunnableConfig(
    thread_id=thread_id,
    metadata={"persistent_context": True}
)

# All calls with same thread_id share context
```

### Checkpointing:
```python
# Save conversation state at specific points
checkpoint_config = RunnableConfig(
    checkpoint_id=f"before_complex_operation_{timestamp}"
)

# If something goes wrong, can restore from checkpoint
```

---

## Real-World Config Examples

### 1. Development vs Production

```python
# Development configuration
dev_config = RunnableConfig(
    configurable={"temperature": 0.2},
    callbacks=[StdOutCallbackHandler()],  # Verbose logging
    metadata={"environment": "development"},
    tags=["debug", "testing"],
    recursion_limit=50  # More lenient for debugging
)

# Production configuration  
prod_config = RunnableConfig(
    configurable={"temperature": 0.01},
    callbacks=[MLflowCallbackHandler(), AlertingHandler()],
    metadata={"environment": "production"},
    tags=["production", "monitored"],
    recursion_limit=10  # Strict limits
)
```

### 2. User-Specific Configuration

```python
def get_user_config(user_id, preferences):
    return RunnableConfig(
        configurable={
            "temperature": preferences.get("creativity_level", 0.1),
            "max_tokens": preferences.get("response_length", 1000)
        },
        metadata={
            "user_id": user_id,
            "preferences": preferences,
            "timestamp": datetime.now().isoformat()
        },
        tags=[
            f"user_{user_id}",
            preferences.get("user_tier", "standard"),
            preferences.get("language", "english")
        ]
    )

# Usage
user_config = get_user_config("sourav_banerjee", {
    "creativity_level": 0.3,
    "response_length": 1500,
    "user_tier": "premium",
    "language": "english"
})

response = agent.invoke(messages, config=user_config)
```

### 3. Experiment Configuration

```python
# A/B test different configurations
def get_experiment_config(experiment_group):
    base_config = {
        "callbacks": [ExperimentTracker(experiment_group)],
        "metadata": {"experiment": experiment_group},
        "tags": ["experiment", experiment_group]
    }
    
    if experiment_group == "high_creativity":
        base_config["configurable"] = {"temperature": 0.8}
    elif experiment_group == "high_precision":
        base_config["configurable"] = {"temperature": 0.0}
    else:  # control group
        base_config["configurable"] = {"temperature": 0.1}
        
    return RunnableConfig(**base_config)

# Random assignment for A/B testing
experiment_group = random.choice(["high_creativity", "high_precision", "control"])
config = get_experiment_config(experiment_group)
```

---

## How Config Flows Through the Agent

Let's trace how config flows through our entire agent:

### 1. Initial Call
```python
# User makes a request
user_config = RunnableConfig(
    configurable={"temperature": 0.5},
    metadata={"user_id": "sourav"},
    callbacks=[MyCustomTracker()]
)

response = agent.invoke(messages, config=user_config)
```

### 2. Agent Invocation
```python
# Inside the compiled graph
def agent_node(state, config):  # Config passed through
    return call_model(state, config)
```

### 3. Call Model Execution  
```python
def call_model(state: ChatAgentState, config: RunnableConfig):
    # Config is passed to the model pipeline
    response = model_runnable.invoke(state, config)
    return {"messages": [response]}
```

### 4. Model Pipeline
```python
# Preprocessor gets config
processed_messages = preprocessor.invoke(state, config)

# Model gets config (with all overrides)
response = model.invoke(processed_messages, config)
```

### 5. Tool Execution
```python
# If tools are called, they also get the config
def tools_node(state, config):  # Same config propagated
    return tool_executor.invoke(state, config)
```

**Key insight**: The same `config` object flows through every component, ensuring consistent behavior and monitoring across the entire execution.

---

## Best Practices for Config Usage

### 1. Environment-Specific Configs
```python
class ConfigManager:
    @staticmethod
    def get_config(environment="production"):
        if environment == "production":
            return RunnableConfig(
                configurable={"temperature": 0.01},
                callbacks=[ProductionMonitor()],
                recursion_limit=5
            )
        elif environment == "staging":
            return RunnableConfig(
                configurable={"temperature": 0.1},
                callbacks=[StagingMonitor()],
                recursion_limit=10
            )
        else:  # development
            return RunnableConfig(
                configurable={"temperature": 0.2},
                callbacks=[DebugMonitor()],
                recursion_limit=25
            )
```

### 2. Request-Level Customization
```python
def create_request_config(user_request):
    base_config = get_default_config()
    
    # Adjust based on request type
    if "creative" in user_request.lower():
        base_config.configurable["temperature"] = 0.7
    elif "precise" in user_request.lower():
        base_config.configurable["temperature"] = 0.0
        
    # Add request-specific metadata
    base_config.metadata.update({
        "request_hash": hash(user_request),
        "request_length": len(user_request),
        "timestamp": datetime.now().isoformat()
    })
    
    return base_config
```

### 3. Error Handling with Config
```python
def robust_agent_call(messages, config=None):
    if config is None:
        config = get_default_config()
        
    try:
        return agent.invoke(messages, config=config)
    except RecursionError:
        # Retry with lower recursion limit
        safe_config = RunnableConfig(
            **config.dict(),
            recursion_limit=5
        )
        return agent.invoke(messages, config=safe_config)
    except Exception as e:
        # Log error with config context
        logger.error(f"Agent failed with config {config.dict()}: {e}")
        raise
```

The `config` parameter is what makes LangChain agents truly production-ready, providing the control, observability, and flexibility needed for real-world deployments. It's the bridge between your agent's core logic and the operational requirements of running AI systems at scale.

# Deep Dive: The `config` Parameter in `call_model`

Let me explain the `config: RunnableConfig` parameter in detail, as it's a crucial but often overlooked component that controls how the agent executes.

## What is RunnableConfig?

```python
def call_model(
    state: ChatAgentState,
    config: RunnableConfig,  # ← This parameter
):
    response = model_runnable.invoke(state, config)
    return {"messages": [response]}
```

`RunnableConfig` is LangChain's way of passing **execution-time configuration** and **runtime context** to any runnable component. Think of it as a "control panel" for how your agent should behave during execution.

---

## RunnableConfig Structure

```python
from langchain_core.runnables import RunnableConfig

config = RunnableConfig(
    # Core configuration
    configurable: Dict[str, Any] = {},
    
    # Observability and monitoring
    callbacks: List[BaseCallbackHandler] = [],
    metadata: Dict[str, Any] = {},
    tags: List[str] = [],
    
    # Execution control
    recursion_limit: int = 25,
    max_concurrency: Optional[int] = None,
    
    # Threading and async
    thread_id: Optional[str] = None,
    checkpoint_id: Optional[str] = None,
)
```

Let me break down each component:

---

## 1. Configurable Parameters

```python
config = RunnableConfig(
    configurable={
        "temperature": 0.1,
        "max_tokens": 500,
        "top_p": 0.9,
        "custom_param": "value"
    }
)
```

### What it does:
- **Runtime overrides**: Change model behavior without modifying the base configuration
- **Per-request customization**: Different users can have different settings
- **A/B testing**: Test different configurations dynamically

### Example usage in our Databricks agent:

```python
# Base agent configuration
baseline_config = {
    "endpoint_name": "databricks-meta-llama-3-3-70b-instruct",
    "temperature": 0.01,
    "max_tokens": 1000,
}

# Runtime override for creative tasks
creative_config = RunnableConfig(
    configurable={
        "temperature": 0.8,  # More creative responses
        "max_tokens": 2000   # Longer responses
    }
)

# Runtime override for analytical tasks  
analytical_config = RunnableConfig(
    configurable={
        "temperature": 0.0,  # Deterministic responses
        "max_tokens": 500    # Concise responses
    }
)

# Use different configs for different scenarios
response = agent.invoke(messages, config=creative_config)
```

### How it works in practice:

```python
def _build_agent_from_config(self):
    llm = ChatDatabricks(
        endpoint=self.config.get("endpoint_name"),
        temperature=self.config.get("temperature"),  # Can be overridden
        max_tokens=self.config.get("max_tokens"),    # Can be overridden
    )
    # ...
```

When `call_model` executes:
```python
# The configurable parameters merge with base config
effective_temperature = config.configurable.get("temperature") or base_config["temperature"]
```

---

## 2. Callbacks for Observability

```python
config = RunnableConfig(
    callbacks=[
        StdOutCallbackHandler(),        # Print to console
        MLflowCallbackHandler(),        # Log to MLflow
        CustomMetricsHandler(),         # Custom monitoring
    ]
)
```

### What callbacks enable:

**Real-time monitoring:**
```python
class ConversationMonitor(BaseCallbackHandler):
    def on_llm_start(self, serialized, prompts, **kwargs):
        print(f"🤖 LLM called with {len(prompts)} prompts")
        
    def on_llm_end(self, response, **kwargs):
        print(f"✅ LLM responded with {len(response.generations)} outputs")
        
    def on_tool_start(self, serialized, input_str, **kwargs):
        print(f"🔧 Tool {serialized['name']} called")
        
    def on_tool_end(self, output, **kwargs):
        print(f"✅ Tool completed: {output[:100]}...")
```

**Performance tracking:**
```python
class PerformanceTracker(BaseCallbackHandler):
    def __init__(self):
        self.start_time = None
        self.token_count = 0
        
    def on_llm_start(self, **kwargs):
        self.start_time = time.time()
        
    def on_llm_end(self, response, **kwargs):
        duration = time.time() - self.start_time
        # Log metrics to your monitoring system
        log_metric("llm_duration", duration)
        log_metric("token_count", self.token_count)
```

**Usage in our agent:**
```python
# Monitor a specific conversation
monitoring_config = RunnableConfig(
    callbacks=[ConversationMonitor(), PerformanceTracker()]
)

# Every call_model execution will trigger these callbacks
response = call_model(state, monitoring_config)
```

---

## 3. Metadata for Context

```python
config = RunnableConfig(
    metadata={
        "user_id": "sourav_banerjee",
        "session_id": "session_123",
        "request_type": "documentation_query",
        "priority": "high",
        "source": "web_interface",
        "experiment_group": "feature_test_a"
    }
)
```

### How metadata is used:

**Request tracking:**
```python
def call_model(state: ChatAgentState, config: RunnableConfig):
    user_id = config.metadata.get("user_id", "anonymous")
    session_id = config.metadata.get("session_id", "unknown")
    
    # Log request with context
    logger.info(f"Processing request for user {user_id} in session {session_id}")
    
    response = model_runnable.invoke(state, config)
    return {"messages": [response]}
```

**Personalization:**
```python
def personalized_preprocessing(state, config):
    user_id = config.metadata.get("user_id")
    user_preferences = get_user_preferences(user_id)
    
    # Modify system prompt based on user preferences
    if user_preferences.get("technical_level") == "expert":
        system_prompt = "Provide detailed technical responses..."
    else:
        system_prompt = "Provide beginner-friendly responses..."
    
    return enhanced_state
```

**Analytics and A/B testing:**
```python
def analytics_callback(config):
    experiment_group = config.metadata.get("experiment_group")
    
    # Track metrics by experiment group
    track_metric(f"response_time_{experiment_group}", duration)
    track_metric(f"tool_usage_{experiment_group}", tool_count)
```

---

## 4. Tags for Categorization

```python
config = RunnableConfig(
    tags=[
        "production",
        "databricks_docs",
        "technical_support",
        "priority_user"
    ]
)
```

### Tag usage examples:

**Filtering and monitoring:**
```python
class TaggedMetricsHandler(BaseCallbackHandler):
    def on_llm_end(self, response, **kwargs):
        tags = kwargs.get("tags", [])
        
        if "priority_user" in tags:
            # Send to priority monitoring dashboard
            send_to_priority_dashboard(response)
            
        if "production" in tags:
            # Different SLA requirements
            check_response_quality(response)
```

**Configuration routing:**
```python
def get_model_config(tags):
    if "high_accuracy" in tags:
        return {"temperature": 0.0}
    elif "creative" in tags:
        return {"temperature": 0.8}
    else:
        return {"temperature": 0.1}
```

---

## 5. Execution Control Parameters

### Recursion Limit
```python
config = RunnableConfig(
    recursion_limit=10  # Prevent infinite loops
)
```

**Why this matters:**
```python
# Without recursion limit, this could loop forever:
# Agent calls tool → Tool calls another tool → Tool calls agent → Agent calls tool...

# With recursion limit:
# After 10 levels deep, execution stops with an error
```

**Custom recursion handling:**
```python
def safe_call_model(state, config):
    try:
        return model_runnable.invoke(state, config)
    except RecursionError:
        return {
            "role": "assistant", 
            "content": "I apologize, this query is too complex. Could you break it down into smaller parts?"
        }
```

### Max Concurrency
```python
config = RunnableConfig(
    max_concurrency=5  # Limit parallel operations
)
```

**Use case:**
```python
# If agent calls multiple tools simultaneously:
tools = ["search_docs", "query_database", "call_api", "analyze_data"]

# Without limit: All 4 tools run in parallel (might overwhelm system)
# With limit: Only 5 concurrent operations allowed
```

---

## 6. Threading and Checkpointing

```python
config = RunnableConfig(
    thread_id="conversation_abc123",
    checkpoint_id="checkpoint_001"
)
```

### Thread ID usage:
```python
# Maintains conversation context across multiple calls
thread_id = f"user_{user_id}_session_{session_id}"

config = RunnableConfig(
    thread_id=thread_id,
    metadata={"persistent_context": True}
)

# All calls with same thread_id share context
```

### Checkpointing:
```python
# Save conversation state at specific points
checkpoint_config = RunnableConfig(
    checkpoint_id=f"before_complex_operation_{timestamp}"
)

# If something goes wrong, can restore from checkpoint
```

---

## Real-World Config Examples

### 1. Development vs Production

```python
# Development configuration
dev_config = RunnableConfig(
    configurable={"temperature": 0.2},
    callbacks=[StdOutCallbackHandler()],  # Verbose logging
    metadata={"environment": "development"},
    tags=["debug", "testing"],
    recursion_limit=50  # More lenient for debugging
)

# Production configuration  
prod_config = RunnableConfig(
    configurable={"temperature": 0.01},
    callbacks=[MLflowCallbackHandler(), AlertingHandler()],
    metadata={"environment": "production"},
    tags=["production", "monitored"],
    recursion_limit=10  # Strict limits
)
```

### 2. User-Specific Configuration

```python
def get_user_config(user_id, preferences):
    return RunnableConfig(
        configurable={
            "temperature": preferences.get("creativity_level", 0.1),
            "max_tokens": preferences.get("response_length", 1000)
        },
        metadata={
            "user_id": user_id,
            "preferences": preferences,
            "timestamp": datetime.now().isoformat()
        },
        tags=[
            f"user_{user_id}",
            preferences.get("user_tier", "standard"),
            preferences.get("language", "english")
        ]
    )

# Usage
user_config = get_user_config("sourav_banerjee", {
    "creativity_level": 0.3,
    "response_length": 1500,
    "user_tier": "premium",
    "language": "english"
})

response = agent.invoke(messages, config=user_config)
```

### 3. Experiment Configuration

```python
# A/B test different configurations
def get_experiment_config(experiment_group):
    base_config = {
        "callbacks": [ExperimentTracker(experiment_group)],
        "metadata": {"experiment": experiment_group},
        "tags": ["experiment", experiment_group]
    }
    
    if experiment_group == "high_creativity":
        base_config["configurable"] = {"temperature": 0.8}
    elif experiment_group == "high_precision":
        base_config["configurable"] = {"temperature": 0.0}
    else:  # control group
        base_config["configurable"] = {"temperature": 0.1}
        
    return RunnableConfig(**base_config)

# Random assignment for A/B testing
experiment_group = random.choice(["high_creativity", "high_precision", "control"])
config = get_experiment_config(experiment_group)
```

---

## How Config Flows Through the Agent

Let's trace how config flows through our entire agent:

### 1. Initial Call
```python
# User makes a request
user_config = RunnableConfig(
    configurable={"temperature": 0.5},
    metadata={"user_id": "sourav"},
    callbacks=[MyCustomTracker()]
)

response = agent.invoke(messages, config=user_config)
```

### 2. Agent Invocation
```python
# Inside the compiled graph
def agent_node(state, config):  # Config passed through
    return call_model(state, config)
```

### 3. Call Model Execution  
```python
def call_model(state: ChatAgentState, config: RunnableConfig):
    # Config is passed to the model pipeline
    response = model_runnable.invoke(state, config)
    return {"messages": [response]}
```

### 4. Model Pipeline
```python
# Preprocessor gets config
processed_messages = preprocessor.invoke(state, config)

# Model gets config (with all overrides)
response = model.invoke(processed_messages, config)
```

### 5. Tool Execution
```python
# If tools are called, they also get the config
def tools_node(state, config):  # Same config propagated
    return tool_executor.invoke(state, config)
```

**Key insight**: The same `config` object flows through every component, ensuring consistent behavior and monitoring across the entire execution.

---

## Best Practices for Config Usage

### 1. Environment-Specific Configs
```python
class ConfigManager:
    @staticmethod
    def get_config(environment="production"):
        if environment == "production":
            return RunnableConfig(
                configurable={"temperature": 0.01},
                callbacks=[ProductionMonitor()],
                recursion_limit=5
            )
        elif environment == "staging":
            return RunnableConfig(
                configurable={"temperature": 0.1},
                callbacks=[StagingMonitor()],
                recursion_limit=10
            )
        else:  # development
            return RunnableConfig(
                configurable={"temperature": 0.2},
                callbacks=[DebugMonitor()],
                recursion_limit=25
            )
```

### 2. Request-Level Customization
```python
def create_request_config(user_request):
    base_config = get_default_config()
    
    # Adjust based on request type
    if "creative" in user_request.lower():
        base_config.configurable["temperature"] = 0.7
    elif "precise" in user_request.lower():
        base_config.configurable["temperature"] = 0.0
        
    # Add request-specific metadata
    base_config.metadata.update({
        "request_hash": hash(user_request),
        "request_length": len(user_request),
        "timestamp": datetime.now().isoformat()
    })
    
    return base_config
```

### 3. Error Handling with Config
```python
def robust_agent_call(messages, config=None):
    if config is None:
        config = get_default_config()
        
    try:
        return agent.invoke(messages, config=config)
    except RecursionError:
        # Retry with lower recursion limit
        safe_config = RunnableConfig(
            **config.dict(),
            recursion_limit=5
        )
        return agent.invoke(messages, config=safe_config)
    except Exception as e:
        # Log error with config context
        logger.error(f"Agent failed with config {config.dict()}: {e}")
        raise
```

The `config` parameter is what makes LangChain agents truly production-ready, providing the control, observability, and flexibility needed for real-world deployments. It's the bridge between your agent's core logic and the operational requirements of running AI systems at scale.